# Predict Employee Attrition


## Preprocessing Attrition Data

In [1]:
#Load the dataset and analyze

import pandas as pd
import os
import tensorflow as tf
import numpy as np

attrition_data = pd.read_csv("employee_attrition.csv")

print("Data Loaded:\n------------------------\n",attrition_data.dtypes)
attrition_data.head()

Data Loaded:
------------------------
 EmployeeID              int64
TotalMonthsOfExp        int64
TotalOrgsWorked         int64
MonthsInOrg             int64
LastPayIncrementBand    int64
AverageFeedback         int64
LastPromotionYears      int64
Attrition               int64
dtype: object


,EmployeeID,TotalMonthsOfExp,TotalOrgsWorked,MonthsInOrg,LastPayIncrementBand,AverageFeedback,LastPromotionYears,Attrition
0,1,110,4,9,5,4,4,1
1,2,103,3,51,1,4,2,0
2,3,41,4,16,5,4,4,1
3,4,32,4,17,5,2,3,0
4,5,80,3,16,3,4,2,0


In [2]:
#Correlation Analysis of target attribute

attrition_data.corr()['Attrition']

EmployeeID             -0.036630
TotalMonthsOfExp        0.019702
TotalOrgsWorked         0.008706
MonthsInOrg             0.012605
LastPayIncrementBand    0.108528
AverageFeedback        -0.008253
LastPromotionYears      0.765641
Attrition               1.000000
Name: Attrition, dtype: float64

In [3]:
#Convert to Dataframe to numpy array
np_attrition = attrition_data.to_numpy().astype(float)

#Create X_train with the first 7 attributes
X_train = np_attrition[:,1:7]
#Create Y_train with attrition attribute
Y_train=np_attrition[:,7]

#Convert Y_train to one-hot-encoding
Y_train = tf.keras.utils.to_categorical(Y_train,2)

print("X-Train Shape : ", X_train.shape)
print("Y-Train Shape : ", Y_train.shape)

X-Train Shape :  (1000, 6)
Y-Train Shape :  (1000, 2)


## Build Attrition model 

In [4]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup hyperparameters for deep learning
EPOCHS=100
BATCH_SIZE=100
VERBOSE=1
NB_CLASSES=2
N_HIDDEN=128
VALIDATION_SPLIT=0.2


model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(6,),
                              name='Dense-Layer-1',
                              activation='relu'))

model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

model.add(keras.layers.Dense(NB_CLASSES,
                             name='Final',
                             activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Epoch 1/100
8/8 [==============================] - 1s 92ms/step - accuracy: 0.6862 - loss: 0.7466 - val_accuracy: 0.7850 - val_loss: 0.6041
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - accuracy: 0.7525 - loss: 0.5726 - val_accuracy: 0.7850 - val_loss: 0.5725
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - accuracy: 0.7812 - loss: 0.5289 - val_accuracy: 0.7800 - val_loss: 0.5243
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - accuracy: 0.7900 - loss: 0.4928 - val_accuracy: 0.8450 - val_loss: 0.5057
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - accuracy: 0.7987 - loss: 0.4724 - val_accuracy: 0.8400 - val_loss: 0.4546
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - accuracy: 0.8087 - loss: 0.4454 - val_accuracy: 0.8100 - val_loss: 0.4249
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - accuracy: 0.8050 - loss: 0.4327 - val_accuracy: 0.8000 - val_loss: 0.4373
Epoch 8/100
8/8 [=========

##  Predict Attrition 

In [5]:
TotalMonthsOfExp=40
TotalOrgsWorked=4
MonthsInOrg=20
LastPayIncrementBand=5
AverageFeedback=4
LastPromotionYears=4

print("Will employee leave ?", model.predict_classes([[TotalMonthsOfExp,
                                  TotalOrgsWorked,
                                  MonthsInOrg,
                                  LastPayIncrementBand,
                                  AverageFeedback,
                                  LastPromotionYears]]))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Will employee leave ? [1]


In [6]:
#Bulk predictions

print(model.predict_classes(
    [[111,5,85,3,2,2],
    [31,2,15,4,1,4],
    [61,4,24,1,4,3],
    [77,4,35,3,1,1],
    [81,5,7,1,2,3],
    [113,4,112,5,4,1],
    [101,2,48,5,1,4],
    [45,4,22,5,3,1],
    [25,2,2,2,3,2],
    [97,3,15,3,2,4]]))

[0 1 0 0 0 0 1 0 0 1]
